In [ ]:
import sys
sys.path.append('/code')

from database.models import (Protein, Organism, Classification, Molecule, Activity, ActivityType, Source, Quality, CID)

from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session
from sqlalchemy.orm import sessionmaker
import os
import glob
from rdkit import Chem
from razi.rdkit_postgresql.functions import morganbv_fp

import pandas as pd
# import modin.pandas as pd
import numpy as np
import json

import multiprocessing
import gc
from tqdm import tqdm
from copy import copy
import numpy as np

import dask.dataframe as dd


def get_db_session():
    engine = create_engine(
        os.environ.get('SQLALCHEMY_URL'), convert_unicode=True,
        pool_recycle=3600, pool_size=10)
    db_session = scoped_session(sessionmaker(
        autocommit=False, autoflush=False, bind=engine))
    
    return db_session


def get_or_create(session, model, **kwargs):
    instance = session.query(model).filter_by(**kwargs).first()
    if instance:
        return instance
    else:
        instance = model(**kwargs)
        session.add(instance)
        session.flush()
        session.refresh(instance)
        return instance
    
def get_or_instance(session, model, **kwargs):
    instance = session.query(model).filter_by(**kwargs).first()
    if instance:
        return (False, instance)
    else:
        instance = model(**kwargs)
        return (True, instance)
    
def sanitize_and_split(row, length, spl=';'):
    split = [v.rstrip() for v in str(row).split(spl)]
    if len(split)!= length:
        split = [split[0] for i in range(0,length)]
    
    split = [None if x == '' else x for x in split]
    
    return split


class TypeDecoder(json.JSONDecoder):
    """Custom json decoder to support types as values."""

    def __init__(self, *args, **kwargs):
        """Simple json decoder handling types as values."""
        json.JSONDecoder.__init__(self, object_hook=self.object_hook, *args, **kwargs)

    def object_hook(self, obj):
        """Handle types."""
        if '__type__' not in obj:
            return obj
        module = obj['__type__']['module']
        type_ = obj['__type__']['type']
        if module == 'builtins':
            return getattr(__builtins__, type_)
        loaded_module = importlib.import_module(module)
        return getattr(loaded_module, type_)
    


dtype_file = '../.data/papyrus/05.5/data_types.json'
activity_data = '../.data/papyrus/05.5/05.5_combined_set_without_stereochemistry.tsv.xz'
protein_data = '../.data/papyrus/05.5/05.5_combined_set_protein_targets.tsv.xz'


In [ ]:
with open(dtype_file, 'r') as jsonfile:
        dtypes = json.load(jsonfile, cls=TypeDecoder)['papyrus']

In [ ]:
protein_df = pd.read_csv(protein_data, sep='\t', dtype=dtypes)

In [ ]:
organisms = list(set(protein_df['Organism']))
classifications = []
for cstr in protein_df['Classification']:
    classifications.extend(str(cstr).split('->'))

classes = list(set(classifications))

In [ ]:
db_session = get_db_session()

rows = []

for i, row in protein_df.iterrows():
    organism = get_or_create(session=db_session, model=Organism, organism=row['Organism'])
    classifications_list = str(row['Classification']).split('->')
    classifications = [get_or_create(session=db_session, model=Classification, classification=c) for c in classifications_list]
    
    review_mapping = {'reviewed':1, 'Unreviewed':0, 'unreviewed':0}
    
    prot = Protein(
        target_id = row['target_id'],
        HGNC_symbol = str(row['HGNC_symbol']),
        uniprot_id = row['UniProtID'],
        reviewed = review_mapping[row['Status']],
        organism = organism.id,
        length = row['Length'],
        sequence = row['Sequence'], 
        classifications = classifications
    )
    
    rows.append(prot)
    
db_session.add_all(rows)
db_session.commit()
db_session.remove()
    

In [ ]:
def get_cid_arr(sources_list, cids_list, session):
    sources_cids_list = list(zip(sources_list, cids_list))
    cids = [      
        get_or_create(session=session, 
                      model=CID, 
                      cid=c[1], 
                      source=get_or_create(session=session, 
                                           model=Source, 
                                           source=c[0]).source) for c in sources_cids_list]
    return cids


def get_molecule(session, row_tup, cids):
#     out = []
    mol = Chem.MolFromSmiles(row_tup.SMILES)
    fp = morganbv_fp(row_tup.SMILES)

    # change this to use InChI and/or SMILES
    smiles = Chem.CanonSmiles(row_tup.SMILES)
    created, molecule = get_or_instance(session=session,model=Molecule,smiles=smiles,inchi=row_tup.InChI)

    for cid in cids:
        if cid not in molecule.cids:
            molecule.cids.append(cid)

    if created:
        molecule.smiles=smiles
        molecule.mol=mol
        molecule.inchi_key=row_tup.InChIKey
        molecule.inchi=row_tup.InChI
        molecule.inchi_auxinfo=row_tup.InChI_AuxInfo
        molecule.fp=fp
        molecule.connectivity=row_tup.connectivity
        session.add(molecule)
        session.flush()
        session.refresh(molecule)
        
    return molecule

def get_activity_dicts(row):
    slice_list = []
    if ';' in str(row.pchembl_value):

        pchembl_values = [v.rstrip() for v in row.pchembl_value.split(';')]
        length = len(pchembl_values)

        aids = sanitize_and_split(row=row.AID,length=length)        
        doc_ids = sanitize_and_split(row=row.all_doc_ids,length=length)
        years = sanitize_and_split(row=row.all_years,length=length)
        type_IC50s = sanitize_and_split(row=row.type_IC50,length=length)         
        type_EC50s = sanitize_and_split(row=row.type_EC50,length=length)
        type_KDs = sanitize_and_split(row=row.type_KD,length=length)
        type_Kis = sanitize_and_split(row=row.type_Ki,length=length)

        for j in range(0, len(pchembl_values)):
            update_dict = {
                'pchembl_value': pchembl_values[j],
                'AID': aids[j],
                'doc_id': doc_ids[j],
                'Year': years[j],
                'type_IC50': type_IC50s[j],
                'type_EC50': type_EC50s[j],
                'type_KD': type_KDs[j],
                'type_Ki': type_Kis[j]
            }
            row_copy = copy(row._asdict())

            row_copy.update(update_dict)

            slice_list.append(row_copy)

    else:
        slice_list.append(row._asdict())
            
    return slice_list

def process_row(row, session):
    
    activity_type_map = {
        '1000':'IC50',
        '0100':'EC50',
        '0010':'KD',
        '0001':'Ki',
        '0000':'other',
    }
    
    rows = []
    sources_list = row.source.split(';')
    cids_list = row.CID.split(';')
    cids = get_cid_arr(sources_list, cids_list, session)

    rows.extend(cids)

    molecule = get_molecule(session=session, row_tup=row, cids=cids)

    rows.append(molecule)

    qc,quality = get_or_instance(session=session, model=Quality, quality=row.Quality)
    if qc: rows.append(quality)
    qid = quality.id

    tc,target_id = get_or_instance(session=session, model=Protein, target_id=row.target_id)
    if tc: rows.append(target_id)
    tid = target_id.target_id

    molecule_id = molecule.id

    slice_list = get_activity_dicts(row)

    for s in slice_list:

        a = f"{s['type_IC50']}{s['type_EC50']}{s['type_KD']}{s['type_Ki']}"
        activity_type_str = activity_type_map[a]

        activity_type = get_or_create(session=session, model=ActivityType, type=activity_type_str).id

        try:
            y = int(s['Year'])
        except:
            y = None

        if str(s['doc_id']) in ['nan', 'NaN']:
            doc_id = None
        else:
            doc_id = str(s['doc_id'])

        activity = Activity(
            papyrus_activity_id=s['Activity_ID'],
            quality=qid,
            target_id=tid,
            molecule_id = molecule_id,
            accession=s['accession'],
            protein_type=s['Protein_Type'],
            aid = s['AID'],
            doc_id = doc_id,
            year = y,
            type = activity_type, 
            relation = s['relation'],
            pchembl_value = s['pchembl_value'],
            pchembl_value_mean = s['pchembl_value_Mean'],
            pchembl_value_stdev = s['pchembl_value_StdDev'],
            pchembl_value_SEM = s['pchembl_value_SEM'],
            pchembl_value_n = s['pchembl_value_N'],
            pchembl_value_median = s['pchembl_value_Median'],
            pchembl_value_mad = s['pchembl_value_MAD'],   
        )

        rows.append(activity)
        
    return rows
            

def process_activity_frame(df):
#     df_obj = df.select_dtypes(['object'])
    db_session = get_db_session()
    rows = []
    

    print('processing frame')
    
    
    
#     pool = multiprocessing.Pool(5)
#     results = pool.imap_unordered(process_row, list(zip([row for row in df.itertuples()], db_session)), chunksize=1000)

    ddf = dd.from_pandas(df, chunksize=1000)
    
    # do this bit in parallel?
    for row in tqdm(ddf.itertuples()):
        rows.extend(process_row(row, db_session))
        dd.compute()
            
    print('processing complete')
            
    del(df)
    db_session.add_all(rows)
    print('committing data')
    db_session.commit()
    db_session.close()
    db_session.remove()
    gc.collect()
            
    return True


In [ ]:
reader = pd.read_csv(activity_data, sep='\t', compression='xz', chunksize = 10000, iterator=True, dtype=dtypes)

for (i,df) in enumerate(reader):
    # process each data frame
    print(f'processing chunk {i}')
    process_activity_frame(df)
    gc.collect()


In [ ]:
import dask.dataframe as dd

ddata = dd.from_pandas(df)

In [ ]:
url = os.environ.get('SQLALCHEMY_URL')

In [ ]:
url

In [1]:
import sys
sys.path.append('/code')

from database.models import (Protein, Organism, Classification, Molecule, Activity, ActivityType, Source, Quality, CID)

from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session
from sqlalchemy.orm import sessionmaker
import os
import glob

import pandas as pd
import numpy as np
import json

import multiprocessing
import gc
from tqdm import tqdm
from copy import copy
import numpy as np

from rdkit import Chem

def get_db_session():
    engine = create_engine(
        os.environ.get('SQLALCHEMY_URL'), convert_unicode=True,
        pool_recycle=3600, pool_size=10)
    db_session = scoped_session(sessionmaker(
        autocommit=False, autoflush=False, bind=engine))
    
    return db_session


class TypeDecoder(json.JSONDecoder):
    """Custom json decoder to support types as values."""

    def __init__(self, *args, **kwargs):
        """Simple json decoder handling types as values."""
        json.JSONDecoder.__init__(self, object_hook=self.object_hook, *args, **kwargs)

    def object_hook(self, obj):
        """Handle types."""
        if '__type__' not in obj:
            return obj
        module = obj['__type__']['module']
        type_ = obj['__type__']['type']
        if module == 'builtins':
            return getattr(__builtins__, type_)
        loaded_module = importlib.import_module(module)
        return getattr(loaded_module, type_)
    

# get neccessary molecule data as list of tuples
session = get_db_session()
mol_data = session.query(Molecule.id, Molecule.smiles, Molecule.inchi_auxinfo).all()
mol_ids = {}
for t in tqdm(mol_data):
    combo = f'{t[1]}_{t[2]}'
    if not combo in mol_ids.keys():
        mol_ids[combo] = [t[0]]
    else:
        mol_ids[combo].append(t[0])

# add activity types
types = ['EC50', 'IC50', 'KD', 'Ki', 'other']
# atypes = [ActivityType(type=t) for t in types]
# session.add_all(atypes)
# session.commit()

# return ids for activity types as dict
types = dict(session.query(ActivityType.type, ActivityType.id).all())

# add qualities
qualities = ["High","Low","Medium","Medium;Low","Low;Medium"]
# qs = [Quality(quality=q) for q in qualities]
# session.add_all(qs)
# session.commit()

# return ids for qualities as list of tuples
qualities = dict(session.query(Quality.quality,Quality.id).all())

# get all targets - probably don't need this 
targets = session.query(Protein.target_id).all()

pchembl_val_list = lambda pchembl_values: ([v.rstrip() for v in pchembl_values.split(';')])
smiles = lambda smi: (Chem.CanonSmiles(smi))

converters = {
    'SMILES':smiles,
    'pchembl_value':pchembl_val_list
}

dtype_file = '../.data/papyrus/05.5/data_types.json'
activity_data = '../.data/papyrus/05.5/05.5_combined_set_without_stereochemistry.tsv.xz'

with open(dtype_file, 'r') as jsonfile:
        dtypes = json.load(jsonfile, cls=TypeDecoder)['papyrus']

# all columns needed to process activity data
activity_columns = ['Activity_ID', 'SMILES', 'InChI_AuxInfo', 'accession', 'Protein_Type', 'AID', 'doc_id',
                    'Year', 'type_IC50', 'type_EC50', 'type_KD', 'type_Ki', 'relation', 'pchembl_value',
                    'pchembl_value_Mean', 'pchembl_value_StdDev', 'pchembl_value_SEM', 'pchembl_value_N', 
                    'pchembl_value_Median', 'pchembl_value_MAD','Quality'
                   ]

activity_reader = pd.read_csv(activity_data, 
                     sep='\t', 
                     compression='xz', 
                     chunksize = 10000, 
                     iterator=True, 
                     dtype=dtypes,
                     converters=converters,
                     usecols=activity_columns
                    )



/tmp/ipykernel_19435/230964928.py:25: SADeprecationWarning: The create_engine.convert_unicode parameter and corresponding dialect-level parameters are deprecated, and will be removed in a future release.  Modern DBAPIs support Python Unicode natively and this parameter is unnecessary.
  engine = create_engine(
/tmp/ipykernel_19435/230964928.py:55: SAWarning: relationship 'Protein.classification' will copy column protein.target_id to column ProteinClassification.protein_id, which conflicts with relationship(s): 'Classification.protein' (copies protein.target_id to ProteinClassification.protein_id), 'Protein.classifications' (copies protein.target_id to ProteinClassification.protein_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isol

In [140]:
from datetime import datetime

def sanitize_and_split(row, length, spl=';'):
    split = [v.rstrip() for v in str(row).split(spl)]
    if len(split)!= length:
        split = [split[0] for i in range(0,length)]
    
    split = [None if x == '' else x for x in split]
    
    return split

def get_activity_dicts(row):
    slice_list = []
    pchembl_values = [v.rstrip() for v in row.pchembl_value]

    aids = sanitize_and_split(row=row.AID,length=row.pchembl_len)        
    doc_ids = sanitize_and_split(row=row.all_doc_ids,length=row.pchembl_len)
    years = sanitize_and_split(row=row.all_years,length=row.pchembl_len)
    type_IC50s = sanitize_and_split(row=row.type_IC50,length=row.pchembl_len)         
    type_EC50s = sanitize_and_split(row=row.type_EC50,length=row.pchembl_len)
    type_KDs = sanitize_and_split(row=row.type_KD,length=row.pchembl_len)
    type_Kis = sanitize_and_split(row=row.type_Ki,length=row.pchembl_len)

    for j in range(0, row.pchembl_len):
        update_dict = {
            'pchembl_value': pchembl_values[j],
            'AID': aids[j],
            'doc_id': doc_ids[j],
            'Year': years[j],
            'type_IC50': type_IC50s[j],
            'type_EC50': type_EC50s[j],
            'type_KD': type_KDs[j],
            'type_Ki': type_Kis[j]
        }
        row_copy = copy(row._asdict())

        row_copy.update(update_dict)

        slice_list.append(row_copy)
            
    return slice_list

activity_type_map = {
        '1000':'IC50',
        '0100':'EC50',
        '0010':'KD',
        '0001':'Ki',
        '0000':'other',
    }

def get_atype(binstr):
    return activity_type_map[binstr]

activity_columns = ['Activity_ID', 'SMILES', 'InChI_AuxInfo', 'accession', 'Protein_Type', 'AID', 'doc_id',
                    'Year', 'type_IC50', 'type_EC50', 'type_KD', 'type_Ki', 'relation', 'pchembl_value',
                    'pchembl_value_Mean', 'pchembl_value_StdDev', 'pchembl_value_SEM', 'pchembl_value_N', 
                    'pchembl_value_Median', 'pchembl_value_MAD','Quality', 'all_doc_ids', 'all_years',
                    'target_id'
                   ]

activity_reader = pd.read_csv(activity_data, 
                     sep='\t', 
                     compression='xz', 
                     chunksize = 50000, 
                     iterator=True, 
                     dtype=dtypes,
                     converters=converters,
                     usecols=activity_columns,
                     # skiprows=[i for i in range(1,2950000)]
                    )

xlen = lambda x: (len(x))
sval = lambda x: (x[0])


def get_molid(key):
    return mol_ids[key][0]

mid = lambda x: get_molid(f'{x["SMILES"]}_{x["InChI_AuxInfo"]}')

import math

def yval(val):
    if not val:
        return 0
    if math.isnan(float(val)):
        return 0
    else:
        return int(val)
    

yvalx = lambda x: (yval(x))

for i,df in enumerate(activity_reader):
#     if i<59:
#         continue
    print(i*50000)
    now = datetime.now()        
    current_time = now.strftime("%H:%M:%S")
    print("Start Time =", current_time)
    df['pchembl_len'] = df['pchembl_value'].map(xlen)
    multis = df[df['pchembl_len']>1]
    updated_rows = []
    for row in multis.itertuples():
        updated_rows.extend(get_activity_dicts(row))

    multi_expanded = pd.DataFrame(updated_rows)
    singles = df[df['pchembl_len']==1]
    singles['pchembl_value'] = singles['pchembl_value'].map(sval)
    all_expanded = pd.concat([multi_expanded, singles])
    
    all_expanded['atype'] = all_expanded.apply(lambda x: get_atype(f"{x['type_IC50']}{x['type_EC50']}{x['type_KD']}{x['type_Ki']}"), axis=1)
    all_expanded['molecule_id'] = all_expanded.apply(mid, axis=1)
    all_expanded['quality'] = all_expanded.apply(lambda x: (qualities[x['Quality']]), axis=1)
    all_expanded['activity_id'] = all_expanded.apply(lambda x: (types[x['atype']]), axis=1)
    all_expanded['year'] = all_expanded['Year'].map(yvalx)
    
    all_expanded.rename(columns = {
                                    'Activity_ID': 'papyrus_activity_id',
                                    'Protein_Type': 'protein_type',
                                    'AID': 'aid',
                                    'activity_id': 'type',
                                    'pchembl_value_Mean': 'pchembl_value_mean',
                                    'pchembl_value_StdDev': 'pchembl_value_stdev',
                                    'pchembl_value_N': 'pchembl_value_n',
                                    'pchembl_value_Median': 'pchembl_value_median',
                                    'pchembl_value_MAD': 'pchembl_value_mad'
                                   }, inplace = True)
    
    
    to_keep = ['papyrus_activity_id', 
       'target_id', 'accession', 'protein_type', 'aid', 'doc_id', 
        'relation', 'pchembl_value', 'pchembl_value_mean',
       'pchembl_value_stdev', 'pchembl_value_SEM', 'pchembl_value_n',
       'pchembl_value_median', 'pchembl_value_mad',
       'molecule_id', 'quality', 'type', 'year']
    
    to_del = [x for x in all_expanded.columns if x not in to_keep]
    
    all_expanded.drop(to_del, axis=1, inplace=True)
    all_expanded.fillna('', inplace=True)
    all_expanded.replace(r'^\s*$', None, regex=True, inplace=True)
    
    adicts = all_expanded.to_dict('records')
    clean_adicts = [{k:v if not pd.isnull(v) else None for k,v in d.items()} for d in adicts]
    
    activities = [Activity(**d) for d in clean_adicts]
    session.add_all(activities)
    session.commit()
    now = datetime.now()        
    current_time = now.strftime("%H:%M:%S")
    print("End Time =", current_time)

/tmp/ipykernel_19435/1954633759.py:94: ParserWarning: Both a converter and dtype were specified for column SMILES - only the converter will be used.
  for i,df in enumerate(activity_reader):
/tmp/ipykernel_19435/1954633759.py:94: ParserWarning: Both a converter and dtype were specified for column pchembl_value - only the converter will be used.
  for i,df in enumerate(activity_reader):


0
Start Time = 13:01:19


/tmp/ipykernel_19435/1954633759.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  singles['pchembl_value'] = singles['pchembl_value'].map(sval)


End Time = 13:01:33


/tmp/ipykernel_19435/1954633759.py:94: ParserWarning: Both a converter and dtype were specified for column SMILES - only the converter will be used.
  for i,df in enumerate(activity_reader):
/tmp/ipykernel_19435/1954633759.py:94: ParserWarning: Both a converter and dtype were specified for column pchembl_value - only the converter will be used.
  for i,df in enumerate(activity_reader):


...

56700000
Start Time = 00:22:07
End Time = 00:22:25


/tmp/ipykernel_19435/1954633759.py:94: ParserWarning: Both a converter and dtype were specified for column SMILES - only the converter will be used.
  for i,df in enumerate(activity_reader):
/tmp/ipykernel_19435/1954633759.py:94: ParserWarning: Both a converter and dtype were specified for column pchembl_value - only the converter will be used.
  for i,df in enumerate(activity_reader):


56750000
Start Time = 00:22:41
End Time = 00:23:04


/tmp/ipykernel_19435/1954633759.py:94: ParserWarning: Both a converter and dtype were specified for column SMILES - only the converter will be used.
  for i,df in enumerate(activity_reader):
/tmp/ipykernel_19435/1954633759.py:94: ParserWarning: Both a converter and dtype were specified for column pchembl_value - only the converter will be used.
  for i,df in enumerate(activity_reader):


56800000
Start Time = 00:23:12
End Time = 00:23:22


In [133]:
[d for d in clean_adicts if d['doc_id']=='PubChemAID:488837' ]

[{'papyrus_activity_id': 'AAABHMIRDIOYOK_on_O00167_WT',
  'target_id': 'O00167_WT',
  'accession': 'O00167',
  'protein_type': 'WT',
  'aid': 'PubChemAID:488837',
  'doc_id': 'PubChemAID:488837',
  'relation': None,
  'pchembl_value': None,
  'pchembl_value_mean': None,
  'pchembl_value_stdev': None,
  'pchembl_value_SEM': None,
  'pchembl_value_n': None,
  'pchembl_value_median': None,
  'pchembl_value_mad': None,
  'molecule_id': 439200,
  'quality': 17,
  'type': 20,
  'year': 2010},
 {'papyrus_activity_id': 'AAADJTFUPWEOSD_on_O00167_WT',
  'target_id': 'O00167_WT',
  'accession': 'O00167',
  'protein_type': 'WT',
  'aid': 'PubChemAID:488837',
  'doc_id': 'PubChemAID:488837',
  'relation': None,
  'pchembl_value': None,
  'pchembl_value_mean': None,
  'pchembl_value_stdev': None,
  'pchembl_value_SEM': None,
  'pchembl_value_n': None,
  'pchembl_value_median': None,
  'pchembl_value_mad': None,
  'molecule_id': 1499894,
  'quality': 17,
  'type': 20,
  'year': 2010},
 {'papyrus_acti

In [137]:
print(activities[-1].pchembl_value)

None


In [92]:
session.rollback()

In [3]:
xlen = lambda x: (len(x))

df['pchembl_len'] = df['pchembl_value'].map(xlen)

In [18]:
types

{'EC50': 16, 'IC50': 17, 'KD': 18, 'Ki': 19, 'other': 20}

In [20]:
qualities

{'High': 16, 'Low': 17, 'Medium': 18, 'Medium;Low': 19, 'Low;Medium': 20}

In [5]:
activity_type_map = {
        '1000':'IC50',
        '0100':'EC50',
        '0010':'KD',
        '0001':'Ki',
        '0000':'other',
    }
   

# df['atype'] = df.apply(lambda x: get_atype(f"{x['type_IC50']}{x['type_EC50']}{x['type_KD']}{x['type_Ki']}"), axis=1)

multis = df[df['pchembl_len']>1]

In [6]:
multis.head()

,Activity_ID,Quality,SMILES,InChI_AuxInfo,accession,Protein_Type,AID,doc_id,Year,all_doc_ids,...,type_Ki,relation,pchembl_value,pchembl_value_Mean,pchembl_value_StdDev,pchembl_value_SEM,pchembl_value_N,pchembl_value_Median,pchembl_value_MAD,pchembl_len
104,AAALWAVFSMSWAR_on_P08482_WT,Medium,CCCCCCOc1nsnc1C1=CCCN(C)C1C,"""AuxInfo=1/0/N:1,20,18,2,3,4,5,15,14,16,6,19,1...",P08482,WT,CHEMBL751539;CHEMBL751540,DOI:10.1016/S0960-894X(00)80072-4,1994.0,NaN,...,0,=,"[ 7.080, 7.320]",7.200,0.120,0.120,2.0,7.200,0.120,2
107,AAAMYWMNTPTTKE_on_P28845_WT,High,CC1CN(c2ccc(F)cc2C(F)(F)F)CCN1S(=O)(=O)c1ncc(C...,"""AuxInfo=1/1/N:1,7,6,16,17,10,24,3,2,8,11,5,25...",P28845,WT,PubChemAID:725510;CHEMBL2327915;CHEMBL2327915,PMID:24900572,2013.0,PubChemAID:725510;PMID:24900572;PMID:24900572,...,0,=,"[ 7.301, 7.700, 7.300]",7.434,0.188,0.133,3.0,7.301,0.001,3
108,AAAMYWMNTPTTKE_on_P50172_WT,High,CC1CN(c2ccc(F)cc2C(F)(F)F)CCN1S(=O)(=O)c1ncc(C...,"""AuxInfo=1/1/N:1,14,15,16,17,11,24,3,2,12,6,5,...",P50172,WT,CHEMBL2327914;CHEMBL2327914;CHEMBL2327913;CHEM...,PMID:24900572,2013.0,PMID:24900572,...,0,=,"[ 8.000, 7.700, 6.840, 8.000]",7.635,0.475,0.274,4.0,7.850,0.150,4
109,AAANLCCSPMLHMI_on_P29371_WT,High,COc1ccccc1N1CCN(C(=O)c2cc(-c3ccncc3)[nH]n2)CC1,"""AuxInfo=1/1/N:1,6,5,7,4,20,24,21,23,10,27,11,...",P29371,WT,PubChemAID:589933;CHEMBL1763144,PMID:21376585,2011.0,PubChemAID:589933;PMID:21376585,...,0,=,"[ 5.143, 5.140]",5.141,0.002,0.002,2.0,5.141,0.002,2
110,AAAOOWKRAKCLLN_on_Q7ZJM1_WT,High,COC1OC(COC(=O)C=Cc2ccc(O)c(O)c2)C(O)C(OC(=O)C=...,"""AuxInfo=1/0/N:1,19,35,11,27,18,34,10,26,13,29...",Q7ZJM1,WT,CHEMBL702429,PMID:10969990,2000.0,PMID:10969990,...,0,=,"[ 4.460, 4.080]",4.270,0.190,0.190,2.0,4.270,0.190,2


In [7]:
def sanitize_and_split(row, length, spl=';'):
    split = [v.rstrip() for v in str(row).split(spl)]
    if len(split)!= length:
        split = [split[0] for i in range(0,length)]
    
    split = [None if x == '' else x for x in split]
    
    return split

def get_activity_dicts(row):
    slice_list = []
    pchembl_values = [v.rstrip() for v in row.pchembl_value]

    aids = sanitize_and_split(row=row.AID,length=row.pchembl_len)        
    doc_ids = sanitize_and_split(row=row.all_doc_ids,length=row.pchembl_len)
    years = sanitize_and_split(row=row.all_years,length=row.pchembl_len)
    type_IC50s = sanitize_and_split(row=row.type_IC50,length=row.pchembl_len)         
    type_EC50s = sanitize_and_split(row=row.type_EC50,length=row.pchembl_len)
    type_KDs = sanitize_and_split(row=row.type_KD,length=row.pchembl_len)
    type_Kis = sanitize_and_split(row=row.type_Ki,length=row.pchembl_len)

    for j in range(0, row.pchembl_len):
        update_dict = {
            'pchembl_value': pchembl_values[j],
            'AID': aids[j],
            'doc_id': doc_ids[j],
            'Year': years[j],
            'type_IC50': type_IC50s[j],
            'type_EC50': type_EC50s[j],
            'type_KD': type_KDs[j],
            'type_Ki': type_Kis[j]
        }
        row_copy = copy(row._asdict())

        row_copy.update(update_dict)

        slice_list.append(row_copy)
            
    return slice_list

In [8]:
updated_rows = []
for row in multis.itertuples():
    updated_rows.extend(get_activity_dicts(row))

In [9]:
sval = lambda x: (x[0])

multi_expanded = pd.DataFrame(updated_rows)
singles = df[df['pchembl_len']==1]
singles['pchembl_value'] = singles['pchembl_value'].map(sval)
all_expanded = pd.concat([multi_expanded, singles])

/tmp/ipykernel_19435/2504347540.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  singles['pchembl_value'] = singles['pchembl_value'].map(sval)


In [ ]:
all_expanded['atype'] = all_expanded.apply(lambda x: get_atype(f"{x['type_IC50']}{x['type_EC50']}{x['type_KD']}{x['type_Ki']}"), axis=1)

In [14]:
all_expanded['SMILES']

0                            CCCCCCOc1nsnc1C1=CCCN(C)C1C
1                            CCCCCCOc1nsnc1C1=CCCN(C)C1C
2      CC1CN(c2ccc(F)cc2C(F)(F)F)CCN1S(=O)(=O)c1ncc(C...
3      CC1CN(c2ccc(F)cc2C(F)(F)F)CCN1S(=O)(=O)c1ncc(C...
4      CC1CN(c2ccc(F)cc2C(F)(F)F)CCN1S(=O)(=O)c1ncc(C...
                             ...                        
193    CCCCc1nc(Cl)c(C=NNC2=NCCN2)n1Cc1ccc(-c2ccccc2-...
194    CCCCc1nc(Cl)c(C=NNC2=NCCN2)n1Cc1ccc(-c2ccccc2-...
195    CCCCc1nc(Cl)c(C=NNC2=NCCN2)n1Cc1ccc(-c2ccccc2-...
196          O=C(Nc1nn[nH]n1)c1cnc(N2CCOCC2)nc1NC1CCCCC1
199      Cc1ccc(OCC(=O)NN=Cc2ccc(O)cc2)c([N+](=O)[O-])c1
Name: SMILES, Length: 181, dtype: object

In [16]:
def get_molid(key):
    return mol_ids[key][0]

mid = lambda x: get_molid(f'{x["SMILES"]}_{x["InChI_AuxInfo"]}')

all_expanded['mol_id']=all_expanded.apply(mid, axis=1)

In [17]:
all_expanded

,Index,Activity_ID,Quality,SMILES,InChI_AuxInfo,accession,Protein_Type,AID,doc_id,Year,...,relation,pchembl_value,pchembl_value_Mean,pchembl_value_StdDev,pchembl_value_SEM,pchembl_value_N,pchembl_value_Median,pchembl_value_MAD,pchembl_len,mol_id
0,104.0,AAALWAVFSMSWAR_on_P08482_WT,Medium,CCCCCCOc1nsnc1C1=CCCN(C)C1C,"""AuxInfo=1/0/N:1,20,18,2,3,4,5,15,14,16,6,19,1...",P08482,WT,CHEMBL751539,nan,nan,...,=,7.080,7.200,0.120,0.120,2.0,7.200,0.120,2,439224
1,104.0,AAALWAVFSMSWAR_on_P08482_WT,Medium,CCCCCCOc1nsnc1C1=CCCN(C)C1C,"""AuxInfo=1/0/N:1,20,18,2,3,4,5,15,14,16,6,19,1...",P08482,WT,CHEMBL751540,nan,nan,...,=,7.320,7.200,0.120,0.120,2.0,7.200,0.120,2,439224
2,107.0,AAAMYWMNTPTTKE_on_P28845_WT,High,CC1CN(c2ccc(F)cc2C(F)(F)F)CCN1S(=O)(=O)c1ncc(C...,"""AuxInfo=1/1/N:1,7,6,16,17,10,24,3,2,8,11,5,25...",P28845,WT,PubChemAID:725510,PubChemAID:725510,2013,...,=,7.301,7.434,0.188,0.133,3.0,7.301,0.001,3,439226
3,107.0,AAAMYWMNTPTTKE_on_P28845_WT,High,CC1CN(c2ccc(F)cc2C(F)(F)F)CCN1S(=O)(=O)c1ncc(C...,"""AuxInfo=1/1/N:1,7,6,16,17,10,24,3,2,8,11,5,25...",P28845,WT,CHEMBL2327915,PMID:24900572,2013,...,=,7.700,7.434,0.188,0.133,3.0,7.301,0.001,3,439226
4,107.0,AAAMYWMNTPTTKE_on_P28845_WT,High,CC1CN(c2ccc(F)cc2C(F)(F)F)CCN1S(=O)(=O)c1ncc(C...,"""AuxInfo=1/1/N:1,7,6,16,17,10,24,3,2,8,11,5,25...",P28845,WT,CHEMBL2327915,PMID:24900572,2013,...,=,7.300,7.434,0.188,0.133,3.0,7.301,0.001,3,439226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,NaN,AAAXWRLNADMLIF_on_P25095_WT,Low,CCCCc1nc(Cl)c(C=NNC2=NCCN2)n1Cc1ccc(-c2ccccc2-...,"""AuxInfo=1/1/N:1,2,3,26,27,25,28,4,21,36,22,35...",P25095,WT,CHEMBL651552,PMID:1875348,1991.0,...,=,6.540,6.540,0.000,0.000,1.0,6.540,0.000,1,439253
194,NaN,AAAXWRLNADMLIF_on_P29089_WT,Low,CCCCc1nc(Cl)c(C=NNC2=NCCN2)n1Cc1ccc(-c2ccccc2-...,"""AuxInfo=1/1/N:1,2,3,26,27,25,28,4,21,36,22,35...",P29089,WT,CHEMBL651552,PMID:1875348,1991.0,...,=,6.540,6.540,0.000,0.000,1.0,6.540,0.000,1,439253
195,NaN,AAAXWRLNADMLIF_on_P35351_WT,Low,CCCCc1nc(Cl)c(C=NNC2=NCCN2)n1Cc1ccc(-c2ccccc2-...,"""AuxInfo=1/1/N:1,2,3,26,27,25,28,4,21,36,22,35...",P35351,WT,CHEMBL651552,PMID:1875348,1991.0,...,=,6.540,6.540,0.000,0.000,1.0,6.540,0.000,1,439253
196,NaN,AAAYOFQDTAWTQL_on_P48736_WT,High,O=C(Nc1nn[nH]n1)c1cnc(N2CCOCC2)nc1NC1CCCCC1,"""AuxInfo=1/1/N:25,24,26,23,27,14,18,15,17,10,2...",P48736,WT,Sharma2016.assay,PATENT:EP1682532A1,2004.0,...,=,4.544,4.544,0.000,0.000,1.0,4.544,0.000,1,439254


In [ ]:
    'Activity_ID': 'papyrus_activity_id',
    'Protein_Type': 'protein_type',
    'AID': 'aid',
    'activity_id': 'type',
    'pchembl_value_Mean': 'pchembl_value_mean',
    'pchembl_value_StdDev': 'pchembl_value_stdev',
    'pchembl_value_N': 'pchembl_value_n',
    'pchembl_value_Median': 'pchembl_value_median',
    'pchembl_value_MAD': 'pchembl_value_mad'

In [71]:
all_expanded.columns


Index(['Index', 'Activity_ID', 'Quality', 'SMILES', 'InChI_AuxInfo',
       'target_id', 'accession', 'Protein_Type', 'AID', 'doc_id', 'Year',
       'all_doc_ids', 'all_years', 'type_IC50', 'type_EC50', 'type_KD',
       'type_Ki', 'relation', 'pchembl_value', 'pchembl_value_Mean',
       'pchembl_value_StdDev', 'pchembl_value_SEM', 'pchembl_value_N',
       'pchembl_value_Median', 'pchembl_value_MAD', 'pchembl_len', 'atype',
       'molecule_id', 'quality', 'activity_id', 'year'],
      dtype='object')

In [ ]:
def process_row(row):
    
    activity_type_map = {
        '1000':'IC50',
        '0100':'EC50',
        '0010':'KD',
        '0001':'Ki',
        '0000':'other',
    }
    
    mkey = f'{row.SMILES}_{row.InChI_Auxinfo}'
    mol_ids = mol_ids[mkey]
    qid = qualities[row.Quality]

    qc,quality = get_or_instance(session=session, model=Quality, quality=row.Quality)
    if qc: rows.append(quality)
    qid = quality.id

    tc,target_id = get_or_instance(session=session, model=Protein, target_id=row.target_id)
    if tc: rows.append(target_id)
    tid = target_id.target_id

    molecule_id = molecule.id

    slice_list = get_activity_dicts(row)

    for s in slice_list:

        a = f"{s['type_IC50']}{s['type_EC50']}{s['type_KD']}{s['type_Ki']}"
        activity_type_str = activity_type_map[a]

        activity_type = get_or_create(session=session, model=ActivityType, type=activity_type_str).id

        try:
            y = int(s['Year'])
        except:
            y = None

        if str(s['doc_id']) in ['nan', 'NaN']:
            doc_id = None
        else:
            doc_id = str(s['doc_id'])

        activity = Activity(
            papyrus_activity_id=s['Activity_ID'],
            quality=qid,
            target_id=tid,
            molecule_id = molecule_id,
            accession=s['accession'],
            protein_type=s['Protein_Type'],
            aid = s['AID'],
            doc_id = doc_id,
            year = y,
            type = activity_type, 
            relation = s['relation'],
            pchembl_value = s['pchembl_value'],
            pchembl_value_mean = s['pchembl_value_Mean'],
            pchembl_value_stdev = s['pchembl_value_StdDev'],
            pchembl_value_SEM = s['pchembl_value_SEM'],
            pchembl_value_n = s['pchembl_value_N'],
            pchembl_value_median = s['pchembl_value_Median'],
            pchembl_value_mad = s['pchembl_value_MAD'],   
        )

        rows.append(activity)
        
    return rows

In [ ]:
mol_ids

In [9]:
count = 0
dels = []
for k in tqdm(mol_ids.keys()):
    if len(mol_ids[k])>1:
        to_del = mol_ids[k][1:]
        dels.extend(to_del)
#         for i in to_del:
#             session.query(CID).filter_by(molecule_id=i).delete()
#             session.query(Molecule).filter_by(id=i).delete()

100%|██████████████████████████████████████████████████████| 1425005/1425005 [00:02<00:00, 665152.22it/s]


In [10]:
cids = []
mols = []
count = 0

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

for group in chunker(dels, 1000):
    count +=1
    print(f'{count*1000}/{len(dels)}')
    cids.extend(session.query(CID).filter(CID.molecule_id.in_(group)).all())
    mols.extend(session.query(Molecule).filter(Molecule.id.in_(group)).all())

1/197698
2/197698
3/197698
4/197698
5/197698
6/197698
7/197698
8/197698
9/197698
10/197698
11/197698
12/197698
13/197698
14/197698
15/197698
16/197698
17/197698
18/197698
19/197698
20/197698
21/197698
22/197698
23/197698
24/197698
25/197698
26/197698
27/197698
28/197698
29/197698
30/197698
31/197698
32/197698
33/197698
34/197698
35/197698
36/197698
37/197698
38/197698
39/197698
40/197698
41/197698
42/197698
43/197698
44/197698
45/197698
46/197698
47/197698
48/197698
49/197698
50/197698
51/197698
52/197698
53/197698
54/197698
55/197698
56/197698
57/197698
58/197698
59/197698
60/197698
61/197698
62/197698
63/197698
64/197698
65/197698
66/197698
67/197698
68/197698
69/197698
70/197698
71/197698
72/197698
73/197698
74/197698
75/197698
76/197698
77/197698
78/197698
79/197698
80/197698
81/197698
82/197698
83/197698
84/197698
85/197698
86/197698
87/197698
88/197698
89/197698
90/197698
91/197698
92/197698
93/197698
94/197698
95/197698
96/197698
97/197698
98/197698
99/197698
100/197698
101/1976

In [ ]:
session.commit()

In [7]:
session.rollback()

In [18]:
len(cids)

207118

In [19]:
[session.delete(cid) for cid in cids]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [15]:
[session.delete(m) for m in mols]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [20]:
session.commit()

KeyboardInterrupt: 

In [21]:
session.rollback()

In [22]:
count = 0
for group in chunker(cids, 1000):
    count +=1
    print(f'{count*1000}/{len(cids)}')
    [session.delete(c) for c in group]
    session.commit()

1000/207118
2000/207118
3000/207118
4000/207118
5000/207118
6000/207118
7000/207118
8000/207118
9000/207118
10000/207118
11000/207118
12000/207118
13000/207118
14000/207118
15000/207118
16000/207118
17000/207118
18000/207118
19000/207118
20000/207118
21000/207118
22000/207118
23000/207118
24000/207118
25000/207118
26000/207118
27000/207118
28000/207118
29000/207118
30000/207118
31000/207118
32000/207118
33000/207118
34000/207118
35000/207118
36000/207118
37000/207118
38000/207118
39000/207118
40000/207118
41000/207118
42000/207118
43000/207118
44000/207118
45000/207118
46000/207118
47000/207118
48000/207118
49000/207118
50000/207118
51000/207118
52000/207118
53000/207118
54000/207118
55000/207118
56000/207118
57000/207118
58000/207118
59000/207118
60000/207118
61000/207118
62000/207118
63000/207118
64000/207118
65000/207118
66000/207118
67000/207118
68000/207118
69000/207118
70000/207118
71000/207118
72000/207118
73000/207118
74000/207118
75000/207118
76000/207118
77000/207118
78000/20

In [ ]:
count = 0
for group in chunker(mols, 100):
    count +=1
    print(f'{count*100}/{len(mols)}')
    [session.delete(c) for c in group]
    session.commit()

100/197698
200/197698
300/197698
400/197698
500/197698
600/197698
700/197698
800/197698
900/197698
1000/197698
1100/197698
1200/197698
1300/197698
1400/197698
1500/197698
1600/197698
1700/197698
1800/197698
1900/197698
2000/197698


In [25]:
session.rollback()

/tmp/ipykernel_19395/1760783211.py:1: SAWarning: Session's state has been changed on a non-active transaction - this state will be discarded.
  session.rollback()
